In [175]:
import pandas as pd
import numpy as np
import pickle 
import warnings
warnings.filterwarnings('ignore')

In [176]:
df = pickle.load(open('dataset_level2.pkl','rb'))

In [177]:
df.isnull().sum()

match_id               0
batting_team           0
bowling_team           0
ball                   0
runs                   0
player_dismissed       0
city                8548
venue                  0
dtype: int64

In [178]:
df[df['city'].isnull()]['venue'].value_counts()

venue
Dubai International Cricket Stadium        2969
Pallekele International Cricket Stadium    2066
Melbourne Cricket Ground                   1453
Sydney Cricket Ground                       749
Adelaide Oval                               498
Harare Sports Club                          372
Sharjah Cricket Stadium                     249
Sylhet International Cricket Stadium        128
Carrara Oval                                 64
Name: count, dtype: int64

In [179]:
cities = np.where(df['city'].isnull(),df['venue'].str.split().apply(lambda x:x[0]),df['city'])

In [180]:
df['city'] = cities

In [181]:
df.isnull().sum()

match_id            0
batting_team        0
bowling_team        0
ball                0
runs                0
player_dismissed    0
city                0
venue               0
dtype: int64

In [182]:
df.drop(labels=['venue'],inplace=True,axis=1)

In [183]:
df['city'].value_counts()

city
Colombo          4086
Mirpur           3420
Johannesburg     3331
Dubai            2969
Auckland         2532
                 ... 
Nairobi           123
Potchefstroom     122
Dharamsala        122
Ahmedabad         121
Carrara            64
Name: count, Length: 86, dtype: int64

In [184]:
cities = (df['city'].value_counts()[df['city'].value_counts() > 600]).index.tolist()

In [185]:
len(cities)

35

In [186]:
df = df[df['city'].isin(cities)]

In [187]:
df['current_score'] = df.groupby('match_id')['runs'].cumsum()

In [188]:
df['ball'] = df['ball'].astype(str)
df['over'] = df['ball'].str.split('.').apply(lambda x:x[0])

In [189]:
df['ball_no'] = df['ball'].str.split('.').apply(lambda x:x[1])

In [190]:
df['balls_bowled'] = (df['over'].astype('int')*6) + (df['ball_no'].astype('int'))

In [191]:
df['balls_left'] = 120 - df['balls_bowled']
df['balls_left'] = np.where(df['balls_left']<0,0,df['balls_left'])

In [192]:
df['player_dismissed'] = np.where(df['player_dismissed']!='0',1,0)
# df['player_dismissed'] = df['player_dismissed'].apply(lambda x:0 if x=='0' else 1)

In [193]:
df['player_dismissed'] = df['player_dismissed'].astype('int')
df['player_dismissed'] = df.groupby('match_id')['player_dismissed'].cumsum()
df['wickets_left'] = 10 - df['player_dismissed']

In [194]:
df['crr'] = (df['current_score']*6)/df['balls_bowled']

In [195]:
df['runs'].fillna(0, inplace=True)  # Replace NaNs with 0, or use df.dropna(subset=['runs']) to drop those rows

In [196]:
df['match_id'] = df['match_id'].astype('int')
df['runs'] = pd.to_numeric(df['runs'], errors='coerce')

In [197]:
groups = df.groupby('match_id')

match_ids = df['match_id'].unique()
last_five = []
# for id in match_ids:
#     last_five.extend(groups.get_group(id).rolling(window=30).sum()['runs'].values.tolist())

for id in match_ids:
    match_runs = groups.get_group(id)[['runs']]  # Select only the 'runs' column
    rolling_sum = match_runs.rolling(window=30).sum()
    last_five.extend(rolling_sum['runs'].values.tolist())


In [198]:
df["last_five"] = last_five

In [199]:
final_df = df.groupby('match_id')['runs'].sum().reset_index(name='total_runs').merge(df,on='match_id')

In [200]:
final_df = final_df[['batting_team','bowling_team','city','current_score','balls_left','wickets_left','last_five','crr','total_runs']]

In [201]:
final_df.isnull().sum()/len(df)*100

batting_team      0.00000
bowling_team      0.00000
city              0.00000
current_score     0.00000
balls_left        0.00000
wickets_left      0.00000
last_five        23.80943
crr               0.00000
total_runs        0.00000
dtype: float64

In [202]:
final_df.dropna(inplace=True)

In [203]:
final_df.isna().sum()

batting_team     0
bowling_team     0
city             0
current_score    0
balls_left       0
wickets_left     0
last_five        0
crr              0
total_runs       0
dtype: int64

In [204]:
final_df = final_df.sample(final_df.shape[0])

In [205]:
final_df.sample(2)

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,last_five,crr,total_runs
29737,Australia,India,Sydney,138,23,7,53.0,8.536082,171
47185,Afghanistan,Sri Lanka,Kolkata,60,47,6,17.0,4.931507,153


In [206]:
from sklearn.model_selection import train_test_split

In [207]:
x = final_df.drop(columns=['total_runs'],axis=1)
y = final_df[['total_runs']]

In [208]:
cat = []
con = []
for i in x.columns:
    if x[i].dtype=='object':
        cat.append(i)
    else:
        con.append(i)

In [209]:
Xcat = x[cat]
Xcon = x[con]

In [212]:
Xcat

,batting_team,bowling_team,city
37759,3,9,4
43416,3,7,16
19479,1,7,22
46012,6,5,15
28964,9,6,31
...,...,...,...
29454,8,1,28
29869,1,4,22
21178,3,4,20
47708,8,9,3


In [213]:
Xcat.isnull().sum()

batting_team    0
bowling_team    0
city            0
dtype: int64

In [211]:
# Xcat_en = pd.get_dummies(Xcat,columns=['batting_team','bowling_team', 'city'],drop_first=True)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in Xcat.columns:
    Xcat[i] = le.fit_transform(Xcat[i].values)

In [214]:
from sklearn.preprocessing import StandardScaler

In [215]:
ss = StandardScaler()
Xcon = pd.DataFrame(ss.fit_transform(Xcon), columns=Xcon.columns)

In [218]:
Xcat.reset_index(drop=True, inplace=True)
Xcon.reset_index(drop=True, inplace=True)

In [219]:
X = Xcon.join(Xcat)

In [220]:
X.isna().sum()

current_score    0
balls_left       0
wickets_left     0
last_five        0
crr              0
batting_team     0
bowling_team     0
city             0
dtype: int64

In [221]:
X

,current_score,balls_left,wickets_left,last_five,crr,batting_team,bowling_team,city
0,0.525572,-0.334545,0.621603,-0.552435,0.475733,3,9,4
1,0.405454,-0.334545,0.129273,0.633184,0.265577,3,7,16
2,0.765806,-0.033204,0.621603,0.633184,1.460949,1,7,22
3,0.141197,-0.296877,0.129273,0.717872,-0.145511,6,5,15
4,0.285337,-0.146207,0.621603,-0.044312,0.341017,9,6,31
...,...,...,...,...,...,...,...,...
38472,-0.387319,0.418806,0.621603,-0.467748,-0.080682,8,1,28
38473,0.597642,-1.163231,-1.347720,-0.721809,-0.446155,1,4,22
38474,-0.963881,1.096822,1.113934,0.548497,-0.213581,3,4,20
38475,0.597642,-1.539906,-1.840051,0.125062,-0.789959,8,9,3


In [225]:
xtrain,xtest,ytrain,ytest = train_test_split(X,y,test_size=0.2,random_state=1)

In [241]:
from sklearn.ensemble import RandomForestRegressor

In [247]:
pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.2/124.9 MB 5.9 MB/s eta 0:00:21
   ---------------------------------------- 0.4/124.9 MB 5.6 MB/s eta 0:00:23
   ---------------------------------------- 0.6/124.9 MB 4.5 MB/s eta 0:00:28
   ---------------------------------------- 0.9/124.9 MB 4.8 MB/s eta 0:00:26
   ---------------------------------------- 1.2/124.9 MB 5.3 MB/s eta 0:00:24
   ---------------------------------------- 1.5/124.9 MB 5.6 MB/s eta 0:00:23
   ---------------------------------------- 1.5/124.9 MB 5.6 MB/s eta 0:00:23
    --------------------------------------- 2.0/124.9 MB 5.7 MB/s eta 0:00:22
    --------------------------------------- 2.1/124.9 MB 5.7 MB/s eta 0:00:22
    --------------------------------------- 2.1/124.9 MB 5.7 MB/s eta 0:00:22
    --------------------------------------- 2.1/124.9 MB 5.7 MB/s eta 0:00:22
    --------------------------------------- 2.1/124.9 MB 5.7 MB/s eta 0

In [248]:
from xgboost import XGBRegressor

In [250]:
xg = XGBRegressor(n_estimators=1000,learning_rate=0.2,max_depth=12,random_state=1)

In [251]:
model = xg.fit(xtrain,ytrain)

In [252]:
tr_pred = model.predict(xtrain)
ts_pred = model.predict(xtest)

In [253]:
from sklearn.metrics import mean_absolute_error, r2_score

In [254]:
tr_err = mean_absolute_error(ytrain,tr_pred)
ts_err = mean_absolute_error(ytest,ts_pred)

print(tr_err)
print(ts_err)

0.007402839114112094
2.082348104821917


In [255]:
tr_scr = r2_score(ytrain,tr_pred)
ts_scr = r2_score(ytest,ts_pred)

print(tr_scr)
print(ts_scr)

0.9998957729728652
0.9748803516588855


In [256]:
pickle.dump(model,open('model.pkl','wb'))